In [1]:
print("lol")

lol


In [10]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
%matplotlib inline

In [46]:
with h5py.File('train_catvnoncat.h5', 'r') as file:
    train_set_x_original = np.array(file['train_set_x'][:])
    train_set_y = np.array(file['train_set_y'][:])
    train_set_y = train_set_y.reshape((1, train_set_y.shape[0]))
    
with h5py.File('test_catvnoncat.h5', 'r') as file:
    classes = np.array(file['list_classes'][:])
    test_set_x_original = np.array(file['test_set_x'][:])
    test_set_y = np.array(file['test_set_y'][:])
    test_set_y = test_set_y.reshape((1, test_set_y.shape[0]))

In [48]:
m_train = train_set_x_original.shape[0]
m_test = test_set_x_original.shape[0]
num_px = train_set_x_original.shape[1]

In [50]:
train_set_x_flatten = train_set_x_original.reshape(train_set_x_original.shape[0], -1).T

In [51]:
test_set_x_flatten = test_set_x_original.reshape(test_set_x_original.shape[0], -1).T

In [54]:
train_set_x = train_set_x_flatten / 255
test_set_x = test_set_x_flatten / 255